In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from scipy import sparse
from sklearn.metrics import mean_squared_error
from mvlearn.semi_supervised import CTRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from scipy import linalg
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneOut
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold

# This is the genomic data which contains aneuploidy score from TCGA

In [2]:
###In this data we have Aneuploidy score and status_1p###
df_tcga_luad_dirk_data_clinical_patient = pd.read_csv("Data/TCGA-LUAD-CLIN-PAN-CAN-2018/data_clinical_patient.txt",sep = "\t",comment='#')
df_tcga_luad_dirk_data_clinical_sample = pd.read_csv("Data/TCGA-LUAD-CLIN-PAN-CAN-2018/data_clinical_sample.txt",sep = "\t",comment='#')
patient_data_tcga = pd.merge(df_tcga_luad_dirk_data_clinical_sample,df_tcga_luad_dirk_data_clinical_patient,on='PATIENT_ID')

In [3]:
new_df_tcga = patient_data_tcga[["SEX","RACE","AGE","AJCC_PATHOLOGIC_TUMOR_STAGE",
                                 "PATH_M_STAGE","PATH_N_STAGE","PATH_T_STAGE",'ANEUPLOIDY_SCORE']]


In [4]:
df = pd.read_csv("Data/patient_features_early_stage.csv",sep=",")
Y = df['relapse?'].values
df.drop(['id','relapse?'], inplace=True, axis=1)
print ("Total number of relapsed patient in the data:",np.count_nonzero(Y))
print ("Total number of survived patient in the data:",len(Y)-np.count_nonzero(Y))
print ("Total number of patient:",len(Y))

Total number of relapsed patient in the data: 491
Total number of survived patient in the data: 857
Total number of patient: 1348


In [5]:
new_df_hosp = df[['diagnosis_stage_t', 'diagnosis_stage_n', 'diagnosis_stage_m','gender','race',
                       'age','tumor_stage']]

We do not need the rows where the Aneuploidy score is not labeled or in other words removing rows which has Aneuploidy  score as nan

In [6]:
new_df_tcga = new_df_tcga[new_df_tcga['ANEUPLOIDY_SCORE'].notna()]
new_df_tcga.shape

(501, 8)

In [7]:
#new_df_tcga.isnull().sum().sum()

cat_cols = []
non_cat_cols = []
for col in list(new_df_tcga.columns):
    if new_df_tcga[col].dtype == object:
        cat_cols.append(col)
    else:
        non_cat_cols.append(col)
        
# fill empty cells of categorical data
new_df_tcga[cat_cols]=new_df_tcga[cat_cols].fillna("na")
new_df_tcga[non_cat_cols]=new_df_tcga[non_cat_cols].fillna(-1)
new_df_tcga.head()


,SEX,RACE,AGE,AJCC_PATHOLOGIC_TUMOR_STAGE,PATH_M_STAGE,PATH_N_STAGE,PATH_T_STAGE,ANEUPLOIDY_SCORE
0,Male,na,70.0,STAGE IV,M1,N2,T2,17.0
1,Male,na,67.0,STAGE IB,M0,N0,T2,24.0
2,Female,na,79.0,STAGE IIIA,M0,N1,T3,17.0
3,Male,na,68.0,STAGE IB,M0,N0,T2,22.0
4,Male,na,66.0,STAGE IIIA,M0,N2,T2,1.0


In [8]:
new_df_tcga.isnull().sum().sum()
new_df_tcga["RACE"].unique()

array(['na', 'White', 'Black or African American', 'Asian',
       'American Indian or Alaska Native'], dtype=object)

In [9]:
###Finding the rows which has all the column having null values##
idx = new_df_tcga.isnull().all(1)
nan_tcga = new_df_tcga.loc[idx]
print (nan_tcga.shape)
print (new_df_tcga.shape)
print ("remaining data after clearing all the nan values:",new_df_tcga.shape[0]-nan_tcga.shape[0])

(0, 8)
(501, 8)
remaining data after clearing all the nan values: 501


In [10]:
new_df_tcga_after_nan = new_df_tcga.drop(list(nan_tcga.index),axis=0)
new_df_tcga_after_nan.head()

,SEX,RACE,AGE,AJCC_PATHOLOGIC_TUMOR_STAGE,PATH_M_STAGE,PATH_N_STAGE,PATH_T_STAGE,ANEUPLOIDY_SCORE
0,Male,na,70.0,STAGE IV,M1,N2,T2,17.0
1,Male,na,67.0,STAGE IB,M0,N0,T2,24.0
2,Female,na,79.0,STAGE IIIA,M0,N1,T3,17.0
3,Male,na,68.0,STAGE IB,M0,N0,T2,22.0
4,Male,na,66.0,STAGE IIIA,M0,N2,T2,1.0


In [11]:
new_df_tcga_after_nan.isnull().sum().sum()

0

In [12]:
def normalize_tcga_data_same_as_hosp_data():
    new_df_tcga_after_nan.replace(['STAGE IV', 'STAGE IB', 'STAGE IIIA', 'STAGE IA', 'STAGE IIIB','STAGE IIA'], ['IV','IB',
                                  'IIIA','IA','IIIB','IIA'],inplace=True)
    
    new_df_tcga_after_nan['PATH_M_STAGE'] = new_df_tcga_after_nan.PATH_M_STAGE.replace(['M1', 'M0','MX', 'M1B', 'M1A'],
                                                                                        ['M1','M0','Mx','M1b','M1a'])
    
    new_df_tcga_after_nan['PATH_N_STAGE'] = new_df_tcga_after_nan.PATH_N_STAGE.replace(['N2', 'N0', 'N1', 'N3', 'NX'],
                                                                                        ['N2', 'N0', 'N1', 'N3', 'Nx'])
    
    new_df_tcga_after_nan['PATH_T_STAGE'] = new_df_tcga_after_nan.PATH_T_STAGE.replace(['T2', 'T3', 'T1', 'T4', 'T2B','T1B', 'T2A', 'T1A', 'TX'],
                                                                                        ['T2', 'T3', 'T1', 'T4', 'T2b', 'T1b', 'T2a', 'T1a', 'Tx'])
    
    
    new_df_tcga_after_nan['RACE'] = new_df_tcga_after_nan.RACE.replace(['White', 'American Indian or Alaska Native'],'Caucasian')
    
    new_df_tcga_after_nan['RACE'] = new_df_tcga_after_nan.RACE.replace(['Black or African American'],['African'])
    
    return new_df_tcga_after_nan
        


def rename_the_column_name():
    new_df_tcga_after_nan.rename(columns={'SEX': 'gender', 'RACE': 'race','AGE':'age',
                                                                  'AJCC_PATHOLOGIC_TUMOR_STAGE':'tumor_stage',
                                                                
                                                                  'PATH_M_STAGE':'diagnosis_stage_m',
                                                                  'PATH_N_STAGE':'diagnosis_stage_n',
                                                                  'PATH_T_STAGE':'diagnosis_stage_t'},inplace=True)
    
    
    return new_df_tcga_after_nan
    
    

def reorder_the_column_name():
    new_df_tcga_after_nan = new_df_tcga_after_nan[["diagnosis_stage_t","diagnosis_stage_n",
                                                   "diagnosis_stage_m","gender","race",
                                                   "age","tumor_stage","last_recorded_timestamp_months",
                                                   "number_of_radiotherapies","ANEUPLOIDY_SCORE"]]
    
    return new_df_tcga_after_nan
    
new_df_tcga_after_nan = normalize_tcga_data_same_as_hosp_data()
new_df_tcga_after_nan = rename_the_column_name()
new_df_tcga_after_nan = new_df_tcga_after_nan[["diagnosis_stage_t","diagnosis_stage_n",
                                                   "diagnosis_stage_m","gender","race",
                                                   "age","tumor_stage","ANEUPLOIDY_SCORE"]]

###fill the missing values with -1###
new_df_tcga_after_nan = new_df_tcga_after_nan.fillna(-1)
new_df_tcga_after_nan.head()



,diagnosis_stage_t,diagnosis_stage_n,diagnosis_stage_m,gender,race,age,tumor_stage,ANEUPLOIDY_SCORE
0,T2,N2,M1,Male,na,70.0,IV,17.0
1,T2,N0,M0,Male,na,67.0,IB,24.0
2,T3,N1,M0,Female,na,79.0,IIIA,17.0
3,T2,N0,M0,Male,na,68.0,IB,22.0
4,T2,N2,M0,Male,na,66.0,IIIA,1.0


# Normalizing tumor stage in TCGA datasets

In [13]:
for i in new_df_tcga_after_nan.index:
    if(new_df_tcga_after_nan.at[i,'tumor_stage']== "STAGE IIB"):
        new_df_tcga_after_nan.at[i,'tumor_stage']='IIB'
       
    if(new_df_tcga_after_nan.at[i,'tumor_stage']== "STAGE II"):
        new_df_tcga_after_nan.at[i,'tumor_stage']='II'
       
    if(new_df_tcga_after_nan.at[i,'tumor_stage']== "STAGE I"):
        new_df_tcga_after_nan.at[i,'tumor_stage']='I'
       

# Dummy coding TNM stages in SLCG and TCGA datasets
This idea is got from the paper: https://pubmed.ncbi.nlm.nih.gov/31911366/ (Table 3)


In [14]:

def tnm_staging_uniformity():
    for i in new_df_tcga_after_nan.index:     
        if (new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T1" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T1a" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T1b" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T1c" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T1mi" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="Tis"):
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"] = "T1"

        if (new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T2" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T2a" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"]=="T2b"):
            new_df_tcga_after_nan.at[i, "diagnosis_stage_t"] = "T2"


        if (new_df_tcga_after_nan.at[i, "diagnosis_stage_m"]=="M1" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_m"]=="M1a" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_m"]=="M1b" or 
            new_df_tcga_after_nan.at[i, "diagnosis_stage_m"]=="M1c"):
            new_df_tcga_after_nan.at[i, "diagnosis_stage_m"] = "M1"


    for i in new_df_hosp.index:
        if (new_df_hosp.at[i, "diagnosis_stage_t"]=="T2" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="T2a" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="T2b"):
            new_df_hosp.at[i, "diagnosis_stage_t"] = "T2"

        if (new_df_hosp.at[i, "diagnosis_stage_t"]=="T1" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="T1a" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="T1b" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="T1c" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="T1mi" or 
            new_df_hosp.at[i, "diagnosis_stage_t"]=="Tis"):
            new_df_hosp.at[i, "diagnosis_stage_t"] = "T1"

        if (new_df_hosp.at[i, "diagnosis_stage_m"]=="M1" or
            new_df_hosp.at[i, "diagnosis_stage_m"]=="M1a" or 
            new_df_hosp.at[i, "diagnosis_stage_m"]=="M1b" or 
            new_df_hosp.at[i, "diagnosis_stage_m"]=="M1c"):
            new_df_hosp.at[i, "diagnosis_stage_m"] = "M1"


def tumor_stage_uniformity():
    for i in new_df_tcga_after_nan.index:     
        if (new_df_tcga_after_nan.at[i, "tumor_stage"]=="0"):
            new_df_tcga_after_nan.at[i, "tumor_stage"] = "overall_stage_tumor_0"

        if (new_df_tcga_after_nan.at[i, "tumor_stage"]=="IA" or 
            new_df_tcga_after_nan.at[i, "tumor_stage"]=="I" or
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IA1" or 
            new_df_tcga_after_nan.at[i, "tumor_stage"]=="IA2" or 
            new_df_tcga_after_nan.at[i, "tumor_stage"]=="IA3" or 
            new_df_tcga_after_nan.at[i, "tumor_stage"]=="IB" ):
            new_df_tcga_after_nan.at[i, "tumor_stage"] = "overall_stage_tumor_1"


        if (new_df_tcga_after_nan.at[i, "tumor_stage"]=="IIA" or 
            new_df_tcga_after_nan.at[i, "tumor_stage"]=="II" or
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IIB"):
            new_df_tcga_after_nan.at[i, "tumor_stage"] = "overall_stage_tumor_2"

        if (new_df_tcga_after_nan.at[i, "tumor_stage"]=="III" or 
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IIIA" or 
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IIIB" or 
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IIIC"):   
            new_df_tcga_after_nan.at[i, "tumor_stage"] = "overall_stage_tumor_3"

        if (new_df_tcga_after_nan.at[i, "tumor_stage"]=="IV" or 
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IVA" or 
            new_df_tcga_after_nan.at[i,"tumor_stage"]=="IVB"):
            new_df_tcga_after_nan.at[i, "tumor_stage"] = "overall_stage_tumor_4"

    for i in new_df_hosp.index:
        if (new_df_hosp.at[i, "tumor_stage"]=="0"):
            new_df_hosp.at[i, "tumor_stage"] = "overall_stage_tumor_0"

        if (new_df_hosp.at[i, "tumor_stage"]=="IA" or 
            new_df_hosp.at[i, "tumor_stage"]=="I" or
            new_df_hosp.at[i,"tumor_stage"]=="IA1" or 
            new_df_hosp.at[i, "tumor_stage"]=="IA2" or 
            new_df_hosp.at[i, "tumor_stage"]=="IA3" or
            new_df_hosp.at[i, "tumor_stage"]=="IB"):
            new_df_hosp.at[i, "tumor_stage"] = "overall_stage_tumor_1"


        if (new_df_hosp.at[i, "tumor_stage"]=="IIA" or 
            new_df_hosp.at[i,"tumor_stage"]=="II" or
            new_df_hosp.at[i,"tumor_stage"]=="IIB"):
            new_df_hosp.at[i, "tumor_stage"] = "overall_stage_tumor_2"

        if (new_df_hosp.at[i, "tumor_stage"]=="III" or 
            new_df_hosp.at[i,"tumor_stage"]=="IIIA" or 
            new_df_hosp.at[i,"tumor_stage"]=="IIIB" or 
            new_df_hosp.at[i,"tumor_stage"]=="IIIC"): 
            new_df_hosp.at[i, "tumor_stage"] = "overall_stage_tumor_3"

        if (new_df_hosp.at[i, "tumor_stage"]=="IV" or 
            new_df_hosp.at[i,"tumor_stage"]=="IVA" or
            new_df_hosp.at[i,"tumor_stage"]=="IVB"):
            new_df_hosp.at[i, "tumor_stage"] = "overall_stage_tumor_4"


tnm_staging_uniformity()
tumor_stage_uniformity()   


# Training a machine learning Regression Model to find out which predicts the best Aneuploidy score for the TCGA data##



In [15]:
train_aneuploidy_score = new_df_tcga_after_nan["ANEUPLOIDY_SCORE"].values
train_data_tcga = new_df_tcga_after_nan.drop(['ANEUPLOIDY_SCORE'], axis = 1)

In [16]:

#new_data_all_norm = new_data_all.copy()
#scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
#new_data_all_norm[['age','last_recorded_timestamp_months','number_of_radiotherapies']] = scaler.fit_transform(new_data_all_norm[['age','last_recorded_timestamp_months','number_of_radiotherapies']])
##one hot encoding for categorical variable###
train_data_tcga = pd.get_dummies(train_data_tcga, columns=["diagnosis_stage_t","diagnosis_stage_n","diagnosis_stage_m","gender","race","tumor_stage"])
train_data_tcga.head()

,age,diagnosis_stage_t_T1,diagnosis_stage_t_T2,diagnosis_stage_t_T3,diagnosis_stage_t_T4,diagnosis_stage_t_Tx,diagnosis_stage_n_N0,diagnosis_stage_n_N1,diagnosis_stage_n_N2,diagnosis_stage_n_N3,...,gender_Male,race_African,race_Asian,race_Caucasian,race_na,tumor_stage_na,tumor_stage_overall_stage_tumor_1,tumor_stage_overall_stage_tumor_2,tumor_stage_overall_stage_tumor_3,tumor_stage_overall_stage_tumor_4
0,70.0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1,67.0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
2,79.0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,68.0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
4,66.0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,1,0


In [ ]:
def linear_regression(X_train, X_test,y_train,ytest):
    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)

def dummy_regression(X_train, X_test,y_train,ytest):
    model = DummyRegressor(strategy="mean")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)
    
def linear_svr(X_train, X_test,y_train,ytest):
    mdl = SVR()  
    param_grid = {'C': [0.01,0.1,1]}
    score = 'neg_mean_squared_error'
    mdl_gscv = GridSearchCV(mdl, param_grid, cv=3, scoring = score,n_jobs=-1)
    mdl_gscv.fit(X_train, y_train)
    model = SVR( C = mdl_gscv.best_params_['C'])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)

def random_forest_regression(X_train, X_test,y_train,ytest):
    param_grid = {'max_depth': range(3,10), 
                  'max_features': ['auto'], 
                  'n_estimators': [100,200]}
    
    
    score = 'neg_mean_squared_error'
    mdl = RandomForestRegressor(random_state=1)
    mdl_gscv = GridSearchCV(mdl, param_grid, cv=3, scoring = score,n_jobs=-1)
    mdl_gscv.fit(X_train, y_train)
    model =  RandomForestRegressor(
                                max_depth= mdl_gscv.best_params_['max_depth'],
                                n_estimators= mdl_gscv.best_params_['n_estimators'],
                                max_features =  mdl_gscv.best_params_['max_features'],
                                random_state=1)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)

def gradient_boosting_machine_regression(X_train, X_test,y_train,ytest):
    param_grid = {'max_depth': range(3,10), 
                  'max_features': ['auto'], 
                  'n_estimators': [100,200]}
    
                
    score = 'neg_mean_squared_error'
    mdl = GradientBoostingRegressor(random_state=1)
    mdl_gscv = GridSearchCV(mdl, param_grid, cv=3, scoring = score,n_jobs=-1)
    mdl_gscv.fit(X_train, y_train)
    model =  GradientBoostingRegressor(
                                max_depth= mdl_gscv.best_params_['max_depth'],
                                n_estimators= mdl_gscv.best_params_['n_estimators'],
                                
                                random_state=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)

def knn_regression(X_train, X_test,y_train,ytest):
    knn_reg = KNeighborsRegressor(n_jobs = -1)
    param_grid = {'n_neighbors': np.arange(3, 21)}
    score = 'neg_mean_squared_error'
    model = GridSearchCV(knn_reg, param_grid, cv=3, scoring = score,n_jobs=-1)
    model.fit(X_train,y_train)
    #knn =  KNeighborsRegressor(knn_gscv.best_params_['n_neighbors'])
    #knn.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)

def bayesian_regression(X_train, X_test,y_train,ytest):
    model = BayesianRidge()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)


def neural_nets_regression(X_train, X_test,y_train,ytest):
    neur_regr = MLPRegressor(random_state=1,shuffle=True)
    X_train = preprocessing.scale(X_train)
    X_test =  preprocessing.scale(X_test)
    
    tuned_parameters = [{'hidden_layer_sizes': [2,4,6,8,10,20,30,40],
                       'activation': ['relu'],
                       'solver':['adam'], 'alpha':[0.0001],
                       'batch_size':['auto'], 'learning_rate':['constant'],
                       'learning_rate_init':[0.001], 'max_iter':[500]}]
    

    model = GridSearchCV(MLPRegressor(), tuned_parameters, cv=3,n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_squared_error(y_test, y_pred, squared=False)


list_of_regression_models = [linear_regression,bayesian_regression,dummy_regression,linear_svr, random_forest_regression,gradient_boosting_machine_regression, knn_regression, neural_nets_regression]
                             
FOLDS = 10
kf = KFold(n_splits=FOLDS)
X = train_data_tcga.values
y = train_aneuploidy_score
model_rmse_matrix = np.zeros((len(list_of_regression_models),FOLDS))
mean_rmse = []
error = []
i = 0
for models in list_of_regression_models:
    score = np.zeros(FOLDS)
    cnt = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        score[cnt] = models(X_train, X_test,y_train,y_test)
        cnt+=1
    mean_rmse.append(np.mean(score))
    error.append(np.std(score))
    model_rmse_matrix[i,:]= score
    i+=1
    print (models.__name__,np.mean(score),np.std(score))
        
    

linear_regression 8.016781401935352 0.5840429218799827
bayesian_regression 7.756797500360044 0.5912508204243764
dummy_regression 7.770691386285135 0.5791910260195583
linear_svr 7.674626662492196 0.6215041282646668
random_forest_regression 7.714758786695889 0.561086707684814
gradient_boosting_machine_regression 7.994048872660395 0.48342962862968
knn_regression 7.858325406907963 0.5964477102996257


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f = plt.figure()
##.__name__  gives the function name very handy function##

index = [i.__name__ .upper() for i in list_of_regression_models]
#print (index)
plotdata = pd.DataFrame(
    {"RMSE": mean_rmse}, 
    index=index)

#Plot a bar chart
plotdata.plot(kind="bar",legend=False,color = "orange",yerr=error,align='center', 
              alpha=1.0, ecolor='black', capsize=10)
plt.title("Regression Models",weight = 'bold')

plt.xticks(rotation=0, horizontalalignment="center",weight = 'bold')
plt.xticks(np.arange(len(plotdata.index)), ['LR', 'BR','DR','SVR','RFR','GBMR','KNNR','NNR'])
#plt.ylim(0,1)
plt.ylabel("RMSE",weight = 'bold')
plt.grid(axis ="y")

for i in range(len(plotdata.index)):
    plt.text(i,mean_rmse[i]+error[i],np.around(mean_rmse[i],decimals=2),ha="center",va="bottom")

plt.savefig('performance_aneuploidy_prediction.eps',bbox_inches='tight')


In [ ]:
from scipy.stats import wilcoxon
###9th row of the matrix is SVR now comparing the significance with other methods###
alpha = 0.05
choice_index = 3
for i in range(model_rmse_matrix.shape[0]):
    if i!=choice_index:
        stat, p = wilcoxon(model_rmse_matrix[choice_index,:], model_rmse_matrix[i,:])
        if p > alpha:
            print('Same distribution (fail to reject H0)=>',list_of_regression_models[choice_index].__name__,":",
                  list_of_regression_models[i].__name__,p)
        else:
            print('Different distribution (reject H0)=>',list_of_regression_models[choice_index].__name__,":",
                  list_of_regression_models[i].__name__,p)
    